In [15]:
import pandas as pd
import numpy as np
from datetime import timedelta
from scipy.fftpack import fft, ifft,rfft
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from joblib import dump, load
import pickle

In [16]:
data = pd.read_csv('test.csv',header=None)

In [17]:
def create_no_meal_feature_matrix(non_meal_data):
    non_meal_data['tau_time'] = (24 - non_meal_data.iloc[:,0:19].idxmax(axis=1)) * 5
    non_meal_data['difference_in_glucose_normalized'] = (non_meal_data.iloc[:,0:19].max(axis=1) - non_meal_data.iloc[:,24]) / (non_meal_data.iloc[:,24])
    power_at_first_max = []
    index_of_first_max = []
    power_at_second_max = []
    index_of_second_max = []
    for i in range(len(non_meal_data)):
        array = abs(rfft(non_meal_data.iloc[:,0:24].iloc[i].values.tolist())).tolist()
        sorted_array = abs(rfft(non_meal_data.iloc[:,0:24].iloc[i].values.tolist())).tolist()
        sorted_array.sort()
        power_at_first_max.append(sorted_array[-2])
        power_at_second_max.append(sorted_array[-3])
        index_of_first_max.append(array.index(sorted_array[-2]))
        index_of_second_max.append(array.index(sorted_array[-3]))
    first_differential = []
    second_differential = []
    for i in range(len(non_meal_data)):
        first_differential.append(np.diff(non_meal_data.iloc[:,0:24].iloc[i].tolist()).max())
        second_differential.append(np.diff(np.diff(non_meal_data.iloc[:,0:24].iloc[i].tolist())).max())
    no_meal_feature_matrix = pd.DataFrame({
        'tau_time': non_meal_data['tau_time'],
        'difference_in_glucose_normalized': non_meal_data['difference_in_glucose_normalized'],
        'power_at_first_max': power_at_first_max,
        'power_at_second_max': power_at_second_max,
        'index_of_first_max': index_of_first_max,
        'index_of_second_max': index_of_second_max,
        'first_differential': first_differential,
        'second_differential': second_differential,
    })
    return no_meal_feature_matrix

In [18]:
dataset = create_no_meal_feature_matrix(data)

In [19]:
from joblib import dump, load
with open('DecisionTreeClassifier.pickle', 'rb') as model:
    file = load(model)
    predictions = file.predict(dataset)    
    model.close()

In [20]:
pd.DataFrame(predictions).to_csv('Result.csv',index=False,header=False)